In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
!pip install praw
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import random
import praw
import os
import tarfile
import shutil



gpt2.download_gpt2(model_name="355M")

gpt2.mount_gdrive()

gpt2.copy_checkpoint_from_gdrive(run_name='run2')


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 514Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 129Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 943Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:05, 260Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 341Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 105Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 176Mit/s]                                                       


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def generate_reply(com):
  tf.reset_default_graph()
  with gpt2.start_tf_sess() as sess:
    gpt2.load_gpt2(sess, run_name='run2')
    pref = com + ' [SEP] '
    text = gpt2.generate(sess, seed=1, run_name='run2', return_as_list=True, prefix=pref, truncate='<|endoftext|>', nsamples=10, batch_size=10, temperature = 0.7, top_k = 40)
    return [re.sub('\n+', '\n',re.sub("&amp;#x200B;", "", t[len(pref):])).strip() for t in text]

In [0]:
reddit = praw.Reddit(client_id='client_id', 
                     client_secret='client_secret',
                     password='reddit_password',
                     username='reddit_username',
                     user_agent='reddit user agent name')

In [0]:
reply_record = []
date = '2019_02_04'

for subreddit_name in ['sciencefiction','artificial','scifi','BurningMan','writing','MachineLearning', 'randonauts']:
  subreddit = reddit.subreddit(subreddit_name)

  for h in subreddit.rising(limit=5):
    already_replied = False
    for c in h.comments:
      c.refresh()
      try:
        if c.author == 'tupperware-party':
          already_replied = True
        for r in c.replies:
          if r.author == 'tupperware-party':
            already_replied = True
        if already_replied:
          break
      except:
        pass
    if already_replied:
      continue
    for c in h.comments:
      try:
        if c.score > 3:
          replies = generate_reply(c.body)
          reply_record.extend([(subreddit_name, h.id, h.title, h.score, c.id, c.score, c.body, r) for r in replies])
      except:
        pass
reply_df = pd.DataFrame(reply_record, columns=['subreddit', 'post_id', 'title', 'post_score', 'comment_id', 'comment_score', 'comment', 'proposed_reply'])
reply_df.to_csv(f'proposed_replies_{date}.csv')
gpt2.copy_file_to_gdrive(f'proposed_replies_{date}.csv')


Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run2/model-1000
Loading checkpoint checkpoint/run2/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/ru